In [1]:

import numpy as np
import cv2

image = cv2.imread('Boss.bmp')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow("gray", gray)

image = cv2.CvtColor(input, gray, cv2.COLORGRAY2BGR);

#Clean up image by removing shadows
planes = cv2.split(image)

r_planes = []
r_n_planes = []

for plane in planes:
    dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    r_planes.append(diff_img)
    r_n_planes.append(norm_img)

result = cv2.merge(r_planes)
result_norm = cv2.merge(r_n_planes)

#output 2 shadow images for potential use
cv2.imshow('shadows_out.png', result)
cv2.imshow('shadows_out_norm.png', result_norm)


result = result_norm.copy()


image = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)
cv2.imshow("HSV",image)
lower = np.array([10, 28, 10])
upper = np.array([145, 255, 255])
mask = cv2.inRange(image, lower, upper)
cv2.imshow("range mask",mask)


kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=40)

cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

boxes = []
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    boxes.append([x,y, x+w,y+h])

boxes = np.asarray(boxes)
left = np.min(boxes[:,0])
top = np.min(boxes[:,1])
right = np.max(boxes[:,2])
bottom = np.max(boxes[:,3])

result[close==0] = (255,255,255)
ROI = result[top:bottom, left:right].copy()
cv2.rectangle(result, (left,top), (right,bottom), (36, 255, 12), 2)

cv2.imshow('result', result)
cv2.imshow('ROI', ROI)
cv2.imshow('close', close)
cv2.imwrite('result.png', result)
cv2.imwrite('ROI.png', ROI)
cv2.imwrite('ROI.png', ROI)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.4-dev) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'
